In [1]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pandas as pd
import re
from sklearn.metrics import accuracy_score

In [2]:
def readFileOfTweets(path):
    ret = None
    with open(path, "r") as f:
        ret = f.read().splitlines()
    return ret

In [3]:
negStrings = readFileOfTweets('data/train_neg_full.txt')
negDf = pd.DataFrame(negStrings, columns=['text'])
negDf['y'] = 0

posStrings = readFileOfTweets('data/train_pos_full.txt')
posDf = pd.DataFrame(posStrings, columns=['text'])
posDf['y'] = 1

allDf = pd.concat([negDf, posDf], ignore_index = True)
allDf

,text,y
0,vinco tresorpack 6 ( difficulty 10 of 10 objec...,0
1,glad i dot have taks tomorrow ! ! #thankful #s...,0
2,1-3 vs celtics in the regular season = were fu...,0
3,<user> i could actually kill that girl i'm so ...,0
4,<user> <user> <user> i find that very hard to ...,0
...,...,...
2499995,a warning sign ? (; rt <user> the negativity y...,1
2499996,<user> ff too thank youuu ) ),1
2499997,i just love shumpa ! that's my girl,1
2499998,the best way to start a day ! no matter what h...,1


In [4]:
def findAccuracy(dataset):
    analyzer = SentimentIntensityAnalyzer()
    actual_y = dataset['y']
    pred_y = []
    for sentence in dataset['text']:
        vs = analyzer.polarity_scores(sentence)
        pred_y.append(1 if vs['compound'] >= 0 else 0)
    print("accuracy : ", accuracy_score(actual_y, pred_y))

In [5]:
findAccuracy(allDf)

accuracy :  0.600116


In [6]:
def readTestFile():
    with open("data/test_data.txt", "r") as f:
        content = f.readlines()
    content = [re.split(",", x, maxsplit=1)[1] for x in content]
    return content

def publishResults(test_pred, file_name):
    with open('data/' + file_name, "w") as f:
        f.write("Id,Prediction\n")
        for pred, index in zip(test_pred, range(1, len(test_pred) +1)):
            f.write(str(index) + "," + str(pred) + "\n")

In [8]:
analyzer = SentimentIntensityAnalyzer()
test_polarity = [analyzer.polarity_scores(sentence) for sentence in readTestFile()]
test_pred = [-1 if x['compound'] <= 0 else 1 for x in test_polarity]
publishResults(test_pred, "vader.csv")